In [ ]:
# STEP 1: MAKING WORLD POINTS and CORNER DETECTION

# first, we create 2D world coordinates of the actual checkerboard based on measurements
# we try to map out each inner corner in the coordinate system (X, Y) on a FLAT plane (Z = 0)

import glob
import numpy as np
import cv2
import scipy
def make_world_points(inner_cols, inner_rows, square_size):
    """
    Generate planar (Z=0) world coordinates for a checkerboard pattern.
    Ordered row-major to match cv2.findChessboardCorners.
    """
    objp = []
    for r in range(inner_rows):
        for c in range(inner_cols):
            objp.append([c * square_size, r * square_size, 0.0])
    return np.asarray(objp, dtype=np.float64)

# This function detects corners of the checkerboard using the cv2 tool
def detect_corners(image_path, pattern_size):
    """
    Returns (ok, corners Nx2) subpixel-refined if found.
    pattern_size: (cols, rows) of INNER corners, e.g., (8,6).
    """
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    assert img is not None, f"Failed to read {image_path}"
    ok, corners = cv2.findChessboardCorners(
        img, pattern_size,
        flags=cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE
    )
    if not ok:
        return False, None
    # Subpixel refine
    corners = cv2.cornerSubPix(
        img, corners, (11,11), (-1,-1),
        (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 40, 1e-3)
    )
    return True, corners.reshape(-1,2)

In [8]:
# STEP 2: Compute homography so we can map real world points to the image plane in terms of (u, v)
# RELATION TO NOTES: Perspective projection on a PLANE to get homogenous coordinates of (u,v)
# [u v 1]^T = H [X Y 1]^T, trying to get H

def normalize_points_2d(pts):
    """Normalize 2D points so centroid is at origin and mean distance is sqrt(2)."""
    pts = np.asarray(pts)
    mean = np.mean(pts, axis=0)
    d = np.mean(np.sqrt(np.sum((pts - mean)**2, axis=1)))
    s = np.sqrt(2) / (d if d > 0 else 1.0)
    T = np.array([[s, 0, -s*mean[0]],
                  [0, s, -s*mean[1]],
                  [0, 0, 1]])
    pts_h = np.c_[pts, np.ones(len(pts))]
    pts_n = (T @ pts_h.T).T[:, :2]
    return pts_n, T

def homography_dlt(world_pts_xy, image_pts):
    """
    Compute H (3x3) s.t. lambda*[u v 1]^T = H * [X Y 1]^T
    world_pts_xy: Nx2 (planar coordinates), image_pts: Nx2 (pixels)
    """
    wp = np.asarray(world_pts_xy)
    ip = np.asarray(image_pts)

    # Normalize for numeric stability (2D -> 2D)
    wp_n, T_w = normalize_points_2d(wp)
    ip_n, T_i = normalize_points_2d(ip)

    N = wp.shape[0]
    A = []
    for i in range(N):
        # X, Y = wp_n[i]
        # u, v = ip_n[i]
        X, Y = wp_n[i]
        u, v = ip_n[i]
        A.append([0, 0, 0, -X, -Y, -1, v*X, v*Y, v])
        A.append([X, Y, 1,  0,  0,  0, -u*X, -u*Y, -u])
    A = np.asarray(A)

    # Solve Ah=0 (SVD)
    _, _, VT = np.linalg.svd(A)
    h = VT[-1, :]
    H_n = h.reshape(3, 3)

    # H = H_n

    # # Denormalize
    H = np.linalg.inv(T_i) @ H_n @ T_w
    if H[2,2] != 0:
        H /= H[2, 2]

    print("H_per_image:", H)
    return H



In [9]:
# STEP 3: now that we have the H matrices from multiple views, we can extract K
# P = K [R | t], where K are intrinsic parameters,
# and [R|t] describes camera pose relative to the world
# Here, Zhang's method solves for K from multiple homographies, then R,t from each H

def v_ij(H, i, j):
    """Helper to form Zhang's linear constraints v_ij from a homography."""
    h = H.T  # columns are h1,h2,h3
    return np.array([
        h[i,0]*h[j,0],
        h[i,0]*h[j,1] + h[i,1]*h[j,0],
        h[i,1]*h[j,1],
        h[i,2]*h[j,0] + h[i,0]*h[j,2],
        h[i,2]*h[j,1] + h[i,1]*h[j,2],
        h[i,2]*h[j,2]
    ])

def intrinsics_from_homographies(H_list):
    """
    Solve for B = K^{-T} K^{-1} (6 unknowns, symmetric) using linear system from multiple H.
    Then recover K (3x3) following Zhang (2000).
    """
    V = []
    for H in H_list:
        V.append(v_ij(H, 0, 1))                 # v01
        V.append(v_ij(H, 0, 0) - v_ij(H, 1, 1)) # v00 - v11
    V = np.vstack(V)

    # Solve V b = 0
    _, _, VT = np.linalg.svd(V)
    b = VT[-1, :]  # 6-vector: [B11, B12, B22, B13, B23, B33]

    B11, B12, B22, B13, B23, B33 = b
    # Recover K from B (see Zhang 2000)
    v0 = (B12*B13 - B11*B23) / (B11*B22 - B12**2)
    lam = B33 - (B13**2 + v0*(B12*B13 - B11*B23)) / B11
    alpha = np.sqrt(abs(lam / B11))
    beta  = np.sqrt(abs(lam * B11 / (B11*B22 - B12**2)))
    gamma = -B12 * alpha**2 * beta / lam
    u0    = gamma * v0 / beta - B13 * alpha**2 / lam

    K = np.array([[alpha, gamma, u0],
                  [0,     beta,  v0],
                  [0,     0,     1]])
    # Normalize K so that K[2,2] = 1
    K = K / K[2,2]
    return K

def extrinsics_from_H(K, H):
    """
    For a planar board at Z=0:
      [r1 r2 t] = lambda * K^{-1} H
      r3 = r1 x r2; enforce orthonormality; det(R)=+1
    Returns R (3x3) and t (3,)
    """
    Kinv = np.linalg.inv(K)
    h1, h2, h3 = H[:,0], H[:,1], H[:,2]
    lam = 1.0 / np.linalg.norm(Kinv @ h1)
    r1 = lam * (Kinv @ h1)
    r2 = lam * (Kinv @ h2)
    r3 = np.cross(r1, r2)
    R = np.column_stack([r1, r2, r3])

    # Orthonormalize R via SVD (closest rotation)
    U, _, Vt = np.linalg.svd(R)
    R = U @ Vt
    if np.linalg.det(R) < 0:
        R[:,2] *= -1  # fix handedness

    t = lam * (Kinv @ h3)
    return R, t

# This section generates K (intrinsics) as well as Rotation (R) and translation (t) which are Extrinsics

In [10]:
# STEP 4: Running the pipeline over several images (PLANAR - Zhang's)

def calibrate_from_planar(images_glob,
                          inner_cols=8, inner_rows=6,
                          square_size=30.0,
                          cam_height=None):
    """
    Full planar Zhang-based camera calibration pipeline.
    Uses 2D-2D homographies from a planar board to compute K and per-image extrinsics.

    Parameters
    ----------
    images_glob : str
        Glob pattern for checkerboard calibration images.
    inner_cols, inner_rows : int
        Number of inner corners along width and height.
    square_size : float
        Checkerboard square size (e.g., mm if you pass 30.0).
    cam_height : float or None
        Optional measured camera-to-plane height (same units as square_size).
        If provided, used to fix translation scale.

    Returns
    -------
    K_avg : (3,3) np.ndarray
        Estimated intrinsic matrix.
    extrinsics : list[(path, R, t)]
        Rotation and translation per image.
    projections : list[(path, P)]
        Projection matrix per image (P = K [R|t]).
    """
    # 1) Collect correspondences for each image, compute H
    obj_xy_full = make_world_points(inner_cols, inner_rows, square_size)  # Nx3
    obj_xy = obj_xy_full[:, :2]  # drop Z=0 for homography
    H_list, corner_sets, paths = [], [], []

    image_paths = sorted(glob.glob(images_glob))
    assert image_paths, f"No images match pattern: {images_glob}"

    for p in image_paths:
        ok, img_pts = detect_corners(p, (inner_cols, inner_rows))
        if not ok:
            print(f"[WARN] Corners not found in {p}, skipping.")
            continue
        H = homography_dlt(obj_xy, img_pts)
        if not np.all(np.isfinite(H)):
            print(f"[WARN] Non-finite homography for {p}, skipping.")
            continue
        H_list.append(H)
        corner_sets.append(img_pts)
        paths.append(p)
        # print(f"[OK] Homography from {p}")

    assert len(H_list) >= 2, "Need at least 2 valid images for intrinsics."

    # 2) Intrinsics from homographies
    K = intrinsics_from_homographies(H_list)
    print("\nEstimated intrinsics K:\n", K)

    # 3) Extrinsics per image from each H
    extrinsics = []
    for p, H in zip(paths, H_list):
        try:
            R, t = extrinsics_from_H(K, H)
        except np.linalg.LinAlgError:
            print(f"[WARN] SVD failed for {p}, skipping extrinsics.")
            continue
        extrinsics.append((p, R, t))

    # 4) Fix translation scale with measured camera height (optional but recommended)
    # World frame assumption: plane is Z=0, +Z points "up" from plane to camera.
    # Camera center C = -R^T t. We want C_z = cam_height.
    if cam_height is not None and extrinsics:
        p0, R0, t0 = extrinsics[0]
        C0 = -R0.T @ t0
        if abs(C0[2]) > 1e-9:
            s = cam_height / C0[2]  # scale to match the measured height
            print(f"\nApplying absolute scale using cam_height = {cam_height:.4f}; scale s = {s:.6f}")
            extrinsics = [(p, R, s*t) for (p, R, t) in extrinsics]
        else:
            print("[WARN] First-view C_z ~ 0; skipping scale fix.")

    # 5) Build P_i = K [R|t]
    projections = []
    for (p, R, t) in extrinsics:
        P = K @ np.hstack([R, t.reshape(3,1)])
        projections.append((p, P))

    return K, extrinsics, projections

In [11]:
# Run pipeline

# Set Constants
images_glob = "selected_photos/*.jpg"   # path to your checkerboard images
inner_cols  = 8
inner_rows  = 6
square_size = 30.0     # 30 mm squares for checkerboard (units: mm)
cam_height  = 373.1    # 37.31 cm distance between camera and checkerboard (units: mm)
# cam_height  = None

# Run all
K_avg, extrinsics, projections = calibrate_from_planar(
    images_glob=images_glob,
    inner_cols=inner_cols,
    inner_rows=inner_rows,
    square_size=square_size,
    cam_height=cam_height
)


H_per_image: [[ 7.84772368e+00 -1.93649096e-01  1.50850108e+03]
 [ 5.07498960e-02  7.96576217e+00  1.16287076e+03]
 [-7.71502578e-05 -1.26210545e-05  1.00000000e+00]]
H_per_image: [[ 7.85159272e+00 -1.90697950e-02  9.25114886e+02]
 [-1.00344695e-01  7.93670096e+00  1.29056922e+03]
 [-6.40900800e-05 -1.46192471e-05  1.00000000e+00]]
H_per_image: [[ 7.83280948e+00 -1.05150822e+00  1.86799953e+03]
 [ 9.73584919e-01  7.95832975e+00  1.28458926e+03]
 [-6.57478903e-05  4.31822044e-06  1.00000000e+00]]
H_per_image: [[ 7.88254988e+00 -7.44636917e-02  1.79820787e+03]
 [-4.09042681e-02  7.99940774e+00  1.19759597e+03]
 [-7.10864863e-05 -6.48701859e-06  1.00000000e+00]]
H_per_image: [[ 7.83422031e+00 -6.11523041e-01  1.22513567e+03]
 [ 4.88502338e-01  7.93212244e+00  1.06251184e+03]
 [-6.99312538e-05 -1.04249178e-05  1.00000000e+00]]
H_per_image: [[ 7.79198723e+00 -9.75405562e-01  1.72239505e+03]
 [ 8.59459937e-01  7.93507636e+00  1.04158120e+03]
 [-7.94426941e-05 -5.75356195e-07  1.00000000e+00]

In [12]:
# STEP 6: Validation - Orthogonality & Reprojection Error

# --- 1) Rotation orthonormality check ---
print("\n=== Rotation Matrix Orthogonality Check ===")
for path, R, t in extrinsics:
    ortho_err = np.linalg.norm(R.T @ R - np.eye(3))
    detR = np.linalg.det(R)
    print(f"{path.split('/')[-1]}: ||RᵀR - I|| = {ortho_err:.6e}, det(R) = {detR:.6f}")

# --- 2) Reprojection error (mean ± std per image) ---
def reprojection_error(P, obj_xy, img_pts):
    # lift planar points to homogeneous 3D with Z=0
    XYZ_h = np.c_[obj_xy, np.zeros((obj_xy.shape[0],1)), np.ones(obj_xy.shape[0])]
    uv_proj_h = (P @ XYZ_h.T).T
    uv = uv_proj_h[:, :2] / uv_proj_h[:, 2, None]
    err = np.linalg.norm(uv - img_pts, axis=1)
    return float(err.mean()), float(err.std())

print("\n=== Reprojection Error per Image ===")
# Recreate the same planar world points used in calibration
obj_xy_full = make_world_points(inner_cols, inner_rows, square_size)
obj_xy = obj_xy_full[:, :2]

# For each projection, re-detect corners and compute error
for (path, P) in projections:
    ok, img_pts = detect_corners(path, (inner_cols, inner_rows))
    if not ok:
        print(f"{path.split('/')[-1]}: corners not found at validation time.")
        continue
    mu, sd = reprojection_error(P, obj_xy, img_pts)
    print(f"{path.split('/')[-1]}: mean = {mu:.2f} px, std = {sd:.2f} px")


=== Rotation Matrix Orthogonality Check ===
PXL_20251015_204036401.jpg: ||RᵀR - I|| = 5.162319e-16, det(R) = 1.000000
PXL_20251015_204038338.jpg: ||RᵀR - I|| = 6.051082e-16, det(R) = 1.000000
PXL_20251015_204042061.jpg: ||RᵀR - I|| = 5.081055e-16, det(R) = 1.000000
PXL_20251015_204051458.jpg: ||RᵀR - I|| = 2.553387e-16, det(R) = 1.000000
PXL_20251015_204055860.jpg: ||RᵀR - I|| = 4.455348e-16, det(R) = 1.000000
PXL_20251015_204057569.jpg: ||RᵀR - I|| = 3.223902e-16, det(R) = 1.000000
PXL_20251015_204059267.jpg: ||RᵀR - I|| = 6.829997e-16, det(R) = 1.000000
PXL_20251015_204101178.jpg: ||RᵀR - I|| = 8.438422e-16, det(R) = 1.000000
PXL_20251015_204102905.jpg: ||RᵀR - I|| = 5.445174e-16, det(R) = 1.000000
PXL_20251015_204104631.jpg: ||RᵀR - I|| = 1.083250e-16, det(R) = 1.000000
PXL_20251015_204108195.jpg: ||RᵀR - I|| = 1.426402e-15, det(R) = 1.000000
PXL_20251015_204109931.jpg: ||RᵀR - I|| = 1.529239e-15, det(R) = 1.000000
PXL_20251015_204112244.jpg: ||RᵀR - I|| = 9.215615e-16, det(R) = 1.